${\LARGE \mbox{IE531: Algorithms for Data Analytics}}$

Finding/Characterizing the general solution to a set of simultaenous equations ${\bf Ax} = {\bf y}$.

In [1]:
# Jupyter Notebook 
# Written by Prof. R.S. Sreenivas for "IE523: Financial Computing" 
# to illustrate the thought-process that goes behind computing the general solution to 
# a system of linear equations Ax = b (that is known to have at least one solution)
#
# You should be able to translate/transliterate this Python-Code to C++ Code, using 
# the help of NEWMAT (and other material) to write a generic solver for Ax = b in C++
#
import sys
import numpy as np 
from numpy import matrix
from numpy import linalg
from numpy.linalg import matrix_rank
import math
import matplotlib.pyplot as plt
from sympy import *

In [2]:
# check if there is a solution to Ax = y
def check_if_there_is_solution(A, y) :
    print("Checking if there is solution to Ax = y, where A = ")
    print(A)
    print("and y = ")
    print(y)
    if (matrix_rank(A) == matrix_rank(np.concatenate((A.transpose(), y.transpose())))) :
        print ("There is a solution")
        print ("Rank(A) = Rank ([A | y]) = ", matrix_rank(A))
        return True 
    else :
        print ("There is no solution")
        print ("Rank(A) = ", matrix_rank(A), "; Rank ([A | y]) = ", matrix_rank(np.concatenate((A.transpose(), y.transpose()))))
        return False

${\bf Orthogonal}$ ${\bf Projection}$:  See equation 1 of Lesson 2 of my notes, where we have shown that ${\bf Xz}$ is the $\textit{Orthogonal}$ $\textit{Projection}$ of the vector ${\bf y}$ on to the column-space (${\bf R}$ in the figure below) of a matrix ${\bf A}$ (whose columns are linearly-independent), where ${\bf z} = \left({\bf X}^T{\bf X}\right)^{-1}{\bf X}^T{\bf y}$.

![](orthogonal.pdf)

This is what the function below computes...

In [3]:
# Orthogonal Projection Formula
# A is expected to have linearly independent columns, and this formula gives the expression for
# the orthogonal-projection of vector-y on to the linear-space generated by the columns of A.  If 
# y lies in this linear-space it will give the (unique) combination of scaled-version-of-cols of A 
# that (vector) add up to vector-y

def find_solution(A, y) :
    # we will assume rank(A) = #columns of A
    return (np.linalg.inv(A.transpose()*A)*A.transpose()*y)

$\textbf{Part #2}$: For this you have to "thin-out" the columns of ${\bf A}$ by eliminating $(\#Cols({\bf A}) - rank({\bf A}))$-many columns.  Technically, there are
\begin{equation}
\left(\begin{array}{c} \#Cols({\bf A})\\ (\#Cols({\bf A}) - rank({\bf A}))
\end{array}\right) = \frac{\#Cols({\bf A})!}{rank({\bf A})! \times (\#Cols({\bf A}) - rank({\bf A}))!} 
\end{equation}
choices for the thinned-out set of columns.  We throw out any member of this collection of $rank({\bf A})$-many-column-matrices that do not have a rank of $rank({\bf A})$. 

In [4]:
def find_general_solution(A) :
    # Trying out all combinations of picking rank(A) columns from the columns of A

    # First, let us get the #columns of A
    _, num_cols = A.shape

    # I need something that help me compute the different ways of picking r objects from k-many objects
    # Googled and got this -- https://www.geeksforgeeks.org/permutation-and-combination-in-python/
    # I adapted the material from this link to my needs... 
    from itertools import combinations
    comb = combinations(list(range(num_cols)), matrix_rank(A))

    # checking if each selection of rank(A)-many columns of A forms a linearly-independent set of vectors
    # if it does, then we use the orthogonal-projection formula to get at the (unique) combination of 
    # scaled-version-of-cols-of-A that (vector) add to vector-y... following this, we have to insert 0's 
    # at the spots that correspond to the deleted columns of A... this gives us what is called a "basic 
    # solution"... following this, we "thin-out" the set of basic-solutions to get at a set of linearly 
    # independent basic-solutions (whose affine combination represents the set-of-all-solutions to the 
    # system of linear equations we set out to solve)

    # This matrix denotes the set of all basic-solutions to Ax = y... it will be filled as progress through
    # the computations described below

    matrix_of_all_basic_solutions = np.empty([1, num_cols])

    for current_choice_of_rank_A_many_columns in list(comb) :
        C = A[:, current_choice_of_rank_A_many_columns]
        if (matrix_rank(C) == matrix_rank(A)) :
            # compute the solution using the Orthogonal Projection formula
            soln_without_inserted_zeros = find_solution(C, y)
 
            # we have to insert/intercalate 0's at the spots corresponding to 
            # the deleted columns of A
            # first, I create a vector of 0's of appropriate size
            # got this from -- https://stackoverflow.com/questions/4056768/how-to-declare-array-of-zeros-in-python-or-an-array-of-a-certain-size
            soln_with_inserted_zeros = [0] * num_cols
            current_index = 0
            for i in range(num_cols) :
                if i in list(current_choice_of_rank_A_many_columns) :
                    soln_with_inserted_zeros[i] = soln_without_inserted_zeros[current_index,0]
                    current_index = current_index + 1
    
            temp = np.array(soln_with_inserted_zeros)
        
            # append this solution (with the zeros appended at appropriate spots) to the matrix of all solutions
            #matrix_of_all_basic_solutions = np.hstack((matrix_of_all_basic_solutions,temp))
            matrix_of_all_basic_solutions = np.append(matrix_of_all_basic_solutions, [temp], axis=0)

    # I figured the empty row, that started the "matrix_of_all_basic_solutions" stayed on as row# 0
    # and I have to delete it... hence the line below
    matrix_of_all_basic_solutions = np.delete(matrix_of_all_basic_solutions,0,0)

    # Transposing the solutions from the row-format to the column-format (my idiosyncrasy! my world is columnar!)
    matrix_of_all_basic_solutions = matrix_of_all_basic_solutions.transpose()
    _, no_of_basic_solns = matrix_of_all_basic_solutions.shape
    print ('\nThere are ' + str(no_of_basic_solns) + '-many Basic Solutions to Ax = y')
    print ('Of these we only need ' + str(matrix_rank(matrix_of_all_basic_solutions)) + 
           '-many of them to construct the General Solution, ')

    # As per theory, there should be rank(A)-many, linearly-independent, basic-solutions... and, the General
    # Solution to Ax = y is the Affine-Combination of these basic-solutions... 

    # now pick rank(matrix_of_all_basic_solutions)-many columns that have a rank that equals 
    # rank(matrix_of_all_basic_solutions) and present their affine-combination as the general solution 
    # to Ax = y...
    soln_combination = combinations(list(range(no_of_basic_solns)), matrix_rank(matrix_of_all_basic_solutions))

    index_for_affine_comb = 1

    for current_choice_of_solutions in list(soln_combination):
        D = matrix_of_all_basic_solutions[:, current_choice_of_solutions]
        if (matrix_rank(D) == matrix_rank(matrix_of_all_basic_solutions)) :
            print ('which is an Affine-Combination of the vectors listed below')
            
            solution_vectors = []
            for i in list(current_choice_of_solutions) :
                globals()['soln%s'%index_for_affine_comb] = matrix_of_all_basic_solutions[:,i].reshape(-1,1)
                #basic_solutions_whose_affine_combination_is_genl_soln[index_for_affine_comb,:] = matrix_of_all_basic_solutions[:,i]
                index_for_affine_comb = index_for_affine_comb + 1
                print(np.around(matrix_of_all_basic_solutions[:,i], decimals=3))
                solution_vectors.append(matrix_of_all_basic_solutions[:,i])
            break
            
    _, number_of_columns = A.shape
    x = [symbols('x%d' % i) for i in range(number_of_columns - matrix_rank(A))]
    
    X = []
    for i in range(number_of_columns) :
        expression1 = ""
        expression2 = "((1"
        index = 1
        for symbol in x:
            expression1 = expression1 + "(" + str(symbol) + "*" + str(solution_vectors[index-1][i].astype(int)) + ") +"
            expression2 = expression2 + "-" + str(symbol) 
            index = index + 1
        expression2 = expression2 + ")*" + str(solution_vectors[index-1][i].astype(int)) + ")"
        expression = expression1 + expression2
        X.append(simplify(expression))

    print ("\nGeneral Solution is:")
    print(np.reshape(X, (number_of_columns,1)))
    
    return (solution_vectors)

$\textbf{Example #1}$: Suppose you are asked to present a general formula for the
solution (if there is a solution, in the first-place) to the
equation:
\begin{equation}\label{chungamaticbunga}
\underbrace{\left(\begin{array}{rrrrrrr}
     1  &   1  &   1  &   1  &  -2  &   3  &   7\\
     1  &  -1  &  -1  &  -1  &   8  &  -3  &   3\\
     1  &   1  &  -1  &  -1  &   8  &   1  &   7\\
     1  &  -1  &  -1  &  -1  &   8  &  -3  &   3\\
     1  &   1  &   1  &  -1  &   8  &   3  &   7\\
     1  &  -1  &   1  &   1  &  -2  &  -1  &   3\\
     1  &   1  &  -1  &   1  &  -2  &   1  &   7\\
     1  &  -1  &  -1  &  -1  &   8  &  -3  &   3
\end{array}\right)}_{\bf A}
\underbrace{\left(\begin{array}{c}
x_{1} \\
x_{2} \\
x_{3} \\
x_{4} \\
x_{5}\\
x_{6}\\
x_{7}
\end{array}\right)}_{\bf x}
= \underbrace{\left(\begin{array}{r}
    57\\
    -1\\
    27\\
    -1\\
    27\\
    29\\
    57\\
    -1
\end{array}\right)}_{\bf y},
\end{equation}
involving eight-equations in seven unknowns.  We check for the existence of a solution by checking
\begin{equation}
rank({\bf A}) = ? = rank({\bf A} \mid {\bf y})
\end{equation}

In [5]:
# The matrix from my notes
A = matrix( [[1,1,1,1,-2,3,7],[1,-1,-1,-1,8,-3,3],[1,1,-1,-1,8,1,7], [1,-1,-1,-1,8,-3,3], 
             [1,1,1,-1,8,3,7], [1,-1,1,1,-2,-1,3], [1,1,-1,1,-2,1,7], [1,-1,-1,-1,8,-3,3]])
y = matrix([[57], [-1], [27], [-1], [27], [29], [57], [-1]])

if (check_if_there_is_solution(A, y)):
    soln = find_general_solution(A)

Checking if there is solution to Ax = y, where A = 
[[ 1  1  1  1 -2  3  7]
 [ 1 -1 -1 -1  8 -3  3]
 [ 1  1 -1 -1  8  1  7]
 [ 1 -1 -1 -1  8 -3  3]
 [ 1  1  1 -1  8  3  7]
 [ 1 -1  1  1 -2 -1  3]
 [ 1  1 -1  1 -2  1  7]
 [ 1 -1 -1 -1  8 -3  3]]
and y = 
[[57]
 [-1]
 [27]
 [-1]
 [27]
 [29]
 [57]
 [-1]]
There is a solution
Rank(A) = Rank ([A | y]) =  4

There are 21-many Basic Solutions to Ax = y
Of these we only need 4-many of them to construct the General Solution, 
which is an Affine-Combination of the vectors listed below
[28. 14.  0. 15.  0.  0.  0.]
[37. 14.  0.  0. -3.  0.  0.]
[28.  0. -7. 15.  0.  7.  0.]
[-7.  0.  0. 15.  0.  0.  7.]

General Solution is:
[[35*x0 + 44*x1 + 35*x2 - 7]
 [14*x0 + 14*x1]
 [-6*x2]
 [x0 - 14*x1 + x2 + 14]
 [-3*x1]
 [7*x2]
 [-7*x0 - 7*x1 - 7*x2 + 7]]
